In [4]:
%run ../yp_utils.py

# Initial setup

In [5]:
paper_pmid = 15883373
paper_name = 'xie_huang_2005' 

In [6]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [7]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
sheets = ['Rapa10nm','Rapa30nm']

In [10]:
original_data_list = []
for s in sheets:
    original_data = pd.read_excel('raw_data/rapa_cellarray_all.xls', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['orf'] = original_data['ORF'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data['data'] = pd.to_numeric(original_data['Rapa/DMSO'], errors='coerce')
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 5960 x 21
Empty DataFrame
Columns: [plate, index, ORF, name, DMSO, Rapa, Rapa/DMSO, Unnamed: 7, Boone and Chan, Unnamed: 9, Unnamed: 10, Unnamed: 11, filtered against our full list, Unnamed: 13, Unnamed: 14, Unnamed: 15, Unnamed: 16, Unnamed: 17, filtered aginst our rapa list, Unnamed: 19, Unnamed: 20, orf]
Index: []

[0 rows x 22 columns]
(5770, 1)
Original data dimensions: 5938 x 7
Empty DataFrame
Columns: [plate, index, ORF, name, DMSO, Rapa, Rapa/DMSO, orf]
Index: []
(5750, 1)


In [11]:
original_data = pd.concat(original_data_list, axis=1)

In [13]:
original_data.index.name='orf'

In [14]:
original_data.head()

,data,data
orf,,
YAL001C,1.135271,0.039321
YAL002W,0.153566,0.253020
YAL003W,1.166415,0.077531
YAL004W,0.557096,0.250771
YAL005C,0.432399,0.343148


# Prepare the final dataset

In [15]:
data = original_data.copy()

In [16]:
dataset_ids = [55,56]
datasets = datasets.reindex(index=dataset_ids)

In [17]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [18]:
data.head()

dataset_id,55,56
data_type,value,value
orf,,
YAL001C,1.135271,0.039321
YAL002W,0.153566,0.253020
YAL003W,1.166415,0.077531
YAL004W,0.557096,0.250771
YAL005C,0.432399,0.343148


## Subset to the genes currently in SGD

In [19]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 15


In [20]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,55,56
,data_type,value,value
gene_id,orf,,
1,YAL001C,1.135271,0.039321
2,YAL002W,0.153566,0.253020
3,YAL003W,1.166415,0.077531
1863,YAL004W,0.557096,0.250771
4,YAL005C,0.432399,0.343148


# Normalize

In [21]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [22]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [23]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,55,56,55,56
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1,YAL001C,1.135271,0.039321,0.403686,-0.093198
2,YAL002W,0.153566,0.253020,-2.387274,-0.065800
3,YAL003W,1.166415,0.077531,0.492229,-0.088299
1863,YAL004W,0.557096,0.250771,-1.240049,-0.066088
4,YAL005C,0.432399,0.343148,-1.594560,-0.054244


# Print out

In [24]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [25]:
from IO.save_data_to_db3 import *

In [26]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 15883373...
Inserting the new data...


100%|██████████| 2/2 [00:18<00:00,  9.14s/it]

Updating the data_modified_on field...
